# Tensor Graph for data visualization


In [1]:
import numpy as np
xy = np.loadtxt ('train-xor.txt', unpack=True)
xdata = xy[0:-1].transpose() # 0 에서 끝-1 까지
ydata = xy[-1:].transpose()
print ('xdata', xdata, '\n', 'ydata', ydata)
print ('xdata[0].shape=', xdata[0].shape, xdata.shape[0], xdata.shape[1])
print ('ydata.shape=', ydata.shape)

xdata [[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  0.]] 
 ydata [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
xdata[0].shape= (2,) 4 2
ydata.shape= (4, 1)


## Specifying Unknown input batch size!

- The input data may have unknown number of data. 
- The codes below shows how to specify such inputs/output data.
- Notice that they are all defined as 2D tensors

In [9]:
import tensorflow as tf

X = tf.placeholder (tf.float32, name='Xinput', shape=[None,xdata.shape[1]])#+list(xdata[0].shape))
print ([None] + list(xdata[0].shape))
Y = tf.placeholder (tf.float32, name='Yinput', shape=[None]+list(ydata[0].shape))
print ([None]+list(ydata[0].shape))

[None, 2]
[None, 1]


# Now, 2-layer network

The network has now twoo layers. This will increase the capacity of the network and result in a correctly fitted model.

In [11]:
nhidd = 12
W1 = tf.Variable (tf.random_uniform([xdata.shape[1],nhidd], -1., 1.), name='W1')
b1 = tf.Variable (tf.zeros([nhidd]), name='b1')
W2 = tf.Variable (tf.random_uniform([nhidd,1],-1.,1.), name='W2')
b2 = tf.Variable (tf.zeros([1]), name='b2')

h1 = tf.sigmoid( tf.matmul (X, W1) + b1)
xor_out = tf.sigmoid (tf.matmul(h1, W2) + b2)

with tf.name_scope ('cost'):
    yxlog = Y * tf.log(xor_out) + (1-Y)*tf.log(1.-xor_out)
    cost = -tf.reduce_mean ( yxlog )
    cost_summ = tf.summary.scalar ('cost', cost)

In [12]:
# minimize
opti = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train2 = opti.minimize(cost)

In [13]:
def evaluate(ss, xor_out, feed_dict):
    pred = ss.run (xor_out, feed_dict={X:xdata, Y:ydata})
    print ('pred=', pred.transpose())
    cpred = ss.run( tf.equal(tf.floor(xor_out+0.5), Y), feed_dict )
    print ('cpred=', cpred.transpose())
    accuracy = tf.reduce_mean (tf.cast(cpred, 'float'))
    print ('accuracy=', accuracy.eval(feed_dict))

### While main learning iteration is proceeding, 

! A directory named ```xor_logs``` will be shown.

! type 

``$ tensorboard --logdir=xor_logs`` 

to see the progress.

In [16]:
feed_dict={X:xdata, Y:ydata}

with tf.Session() as ss:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter ('xor_logs')

    ss.run(tf.global_variables_initializer())

    # data fitting
    # 
    for i in range(3000):
        summary, _ = ss.run ([merged, train2], feed_dict={X:xdata, Y:ydata})
        
        writer.add_summary (summary, i)
        
        if i%1000==0:
            print ('------- data fitting evaluation ------')
            print ('iteraiton: ', i, 
                  ' cost= ', ss.run(cost, feed_dict=feed_dict))
            evaluate (ss, xor_out, feed_dict)            
            
    # model test
    #
    evaluate (ss, xor_out, feed_dict)
print ('finished.') 

------- data fitting evaluation ------
iteraiton:  0  cost=  0.716933
pred= [[ 0.44412249  0.5098983   0.36067724  0.44412249]]
cpred= [[ True  True False  True]]
accuracy= 0.75
------- data fitting evaluation ------
iteraiton:  1000  cost=  0.567355
pred= [[ 0.42388183  0.61890882  0.50321507  0.42388183]]
cpred= [[ True  True  True  True]]
accuracy= 1.0
------- data fitting evaluation ------
iteraiton:  2000  cost=  0.427709
pred= [[ 0.34135488  0.68846655  0.60507303  0.34135488]]
cpred= [[ True  True  True  True]]
accuracy= 1.0
pred= [[ 0.25408694  0.76155657  0.71110827  0.25408694]]
cpred= [[ True  True  True  True]]
accuracy= 1.0
finished.


## EOF